# Register and export agar pad movies

In [1]:
#next two lines make sure that Matplotlib plots are shown properly in Jupyter Notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#next line is required for Napari
%gui qt

#main data analysis packages
import numpy as np
import pandas as pd

#image processing packages
from scipy import ndimage as ndi
import skimage.segmentation as segmentation 
import skimage.filters as filters
from skimage.measure import label, regionprops, regionprops_table
from skimage import morphology

#data plotting packages
import matplotlib
import matplotlib.pyplot as plt
#set default figure size
matplotlib.rc("figure", figsize=(10,5))
import seaborn as sns

#image viewer
import napari
from napari.utils.notebook_display import nbscreenshot

#out of memory computation
from dask_image.imread import imread
import dask.array as da
import dask

#path handling
import pathlib

#file handling
import h5py

#Instead of dask_image.imread.imread() you can also use tifffile.imread() to directly read images into memory
#import tifffile

from glob import glob
import os

from dask.cache import Cache
cache = Cache(2e9)  # Leverage two gigabytes of memory
cache.register()    # Turn cache on globally

import javabridge as jb
import bioformats

import nd2


In [27]:

path = r"/Volumes/Giulia/test.ND2"


from aicsimageio import AICSImage
# Get an AICSImage object
img = AICSImage(path)  # selects the first scene found
img.dask_data  # returns 5D TCZYX dask array
img.dims  # returns a Dimensions object
img.dims.order  # returns string "TCZYX"
img.dims.X  # returns size of X dimension
img.shape  # returns tuple of dimension sizes in TCZYX order

img.get_xarray_dask_stack()

Attempted file (/Volumes/Giulia/test.ND2) load with reader: aicsimageio.readers.nd2_reader.ND2Reader failed with error: [Errno 13] Permission denied


UnsupportedFileFormatError: AICSImage does not support the image: '/Volumes/Giulia/test.ND2'. File extension suggests format: 'nd2'. Install extra format dependency with: `pip install aicsimageio[nd2]`. See all known format extensions and their extra install name with `aicsimageio.formats.FORMAT_IMPLEMENTATIONS`. If the extra dependency is already installed this error may have raised because the file is corrupt or similar issue. For potentially more information and to help debug, try loading the file directly with the desired file format reader instead of with the AICSImage object.

In [25]:
path = "/Volumes/Giulia/test.ND2"
f = nd2.ND2File(path)
raw_data = f.to_dask()
f.close()

PermissionError: [Errno 13] Permission denied

In [11]:
import os

In [23]:
os.path.isdir(path)

False

In [2]:
def _init_logger():
    """This is so that Javabridge doesn't spill out a lot of DEBUG messages
    during runtime.
    From CellProfiler/python-bioformats.
    """
    rootLoggerName = jb.get_static_field("org/slf4j/Logger",
                                         "ROOT_LOGGER_NAME",
                                         "Ljava/lang/String;")

    rootLogger = jb.static_call("org/slf4j/LoggerFactory",
                                "getLogger",
                                "(Ljava/lang/String;)Lorg/slf4j/Logger;",
                                rootLoggerName)

    logLevel = jb.get_static_field("ch/qos/logback/classic/Level",
                                   "WARN",
                                   "Lch/qos/logback/classic/Level;")

    jb.call(rootLogger,
            "setLevel",
            "(Lch/qos/logback/classic/Level;)V",
            logLevel)

In [3]:
jb.start_vm(class_path=bioformats.JARS)
logger = _init_logger()

In [4]:
#set path to ND2 file
path = pathlib.Path('/Volumes/RG van Vliet/Scientific Data/Giulia/Collaborations/Ashley/2023-01-20-agar-pad/2023-01-20-agar-pad.nd2')

#set folder where to store output files
process_dir = pathlib.Path('/Volumes/RG van Vliet/Scientific Data/Giulia/Collaborations/Ashley/2023-01-20-agar-pad-processed/')

In [5]:
def to_str(posixpath):
    return str(posixpath.resolve())   

In [19]:
isinstance(path, pathlib.PurePath)      

False

In [14]:
md.image(0).Pixels.channel_count

3

In [20]:
im = bioformats.ImageReader(str(path.resolve()) )

In [25]:
im.read(series=0, c=0, t=0, rescale=False)

array([[18062, 18418, 18383, ..., 17893, 17934, 17610],
       [17743, 18404, 18488, ..., 17343, 18138, 17576],
       [18088, 18587, 18157, ..., 17789, 18027, 17646],
       ...,
       [16764, 16382, 16892, ..., 17038, 16783, 16653],
       [16367, 16923, 16647, ..., 16661, 16760, 17047],
       [16707, 16460, 16443, ..., 16754, 16611, 16774]], dtype=uint16)

In [27]:
value = dask.delayed(im.read(series=0,))


In [41]:
da.from_delayed(value, (30, 52, 3, 2044, 2048), dtype=np.uint16)

dask.array<from-value, shape=(30, 52, 3, 2044, 2048), dtype=uint16, chunksize=(30, 52, 3, 2044, 2048), chunktype=numpy.ndarray>

In [ ]:
seg_path = root / segment_name #path to Ilastik output
seg_data = h5py.File(seg_path, 'r') #open 
print('keys = ', list(seg_data.keys()))

#we again use Dask to load the data out of memory
#technical comment: we set chunk size to be equal to the size of single frame. 
chuck_size = (1, *im_stack.shape[-2:]) if n_channel==1 else (1, n_channel, *im_stack.shape[-2:])
seg_cell = da.from_array(seg_data['exported_data'], chunks=chuck_size)

#we get a segmentation probability for both labels (cells and BG) but only need the one for the cells, so we extract the correct dimension:
seg_cell = seg_cell[:,fg_idx,:,:]

#add probability layer to Napari Viewer
prop_layer = viewer.add_image(seg_cell, name='probability',colormap='gray')

keys =  ['exported_data']


In [38]:
nd2file(path)

30 52 3 2044 2048


In [37]:
class nd2file:  
    
    def __init__(self, path):
        
        if isinstance(path, pathlib.PurePath):
            path = str(path.resolve()) 
                  
        self.path = path
        self.imhandle = bioformats.ImageReader(path)

        md = bioformats.OMEXML(bioformats.get_omexml_metadata(path))  
        self.p = md.get_image_count()
        self.t = md.image(0).Pixels.SizeT
        self.c = md.image(0).Pixels.channel_count
        self.x = md.image(0).Pixels.SizeX
        self.y = md.image(0).Pixels.SizeY
        
        print(self.p,self.t,self.c,self.y,self.x)
        
#     def _load_im(self):  
        
#         chuck_size = (1, *im_stack.shape[-2:]) if n_channel==1 else (1, n_channel, *im_stack.shape[-2:])
# seg_cell = da.from_array(seg_data['exported_data'], chunks=chuck_size)


        
        
        
#         output =   
#         # Load images:
#         for p, pos in enumerate(self.p):
#             for c, cha in enumerate(self.c):
#                 for f, fra in enumerate(self.t):
#                     # Read frame:
#                     frame = self.filehandle.read(
#                             series=pos, c=cha, t=fra, rescale=False
#                         )
                
#                     # Add to output array:
#                     output[p, f, c, :, :] = frame
                    
#         return output








In [ ]:
from aicsimageio import AICSImage
# Get an AICSImage object
img = AICSImage(path_str)  # selects the first scene found
img.dask_data  # returns 5D TCZYX dask array
img.dims  # returns a Dimensions object
img.dims.order  # returns string "TCZYX"
img.dims.X  # returns size of X dimension
img.shape  # returns tuple of dimension sizes in TCZYX order

img.get_xarray_dask_stack()


In [30]:
#set number of color channels in image
n_channel = 3

#set the sample interval, in each subfolder we will sample images at this interval (e.g. with sample_int=3 we will sample image 0, 3, 6, etc)
#adjust this number to have ~2 images per subfolder
sample_int = 3 

#we will extract two channels here: phase contrast + fluorescence 
ch0 = [] # phase contrast channel
ch1 = [] # fluorescence channel of constitutive marker

for f in folder_names: # loop subfolders
    im_names = [i.name for i in sorted((root / f).glob('*.tif*'))]
    
    for idx, i in enumerate(im_names): #loop images
        if idx%sample_int==0:
            im_path = root / f / i
            im_stack = imread(im_path)
            nfr = im_stack.shape[0]/n_channel
            
            #here we set which frames we sample, adapt to your own images, take frames at start, middle, and end
            frames = [0, max(nfr - 31, 0), max(nfr - 21, 0), nfr-11]
            
            for fr in frames:
                ch0_idx = fr*n_channel
                ch1_idx = fr*n_channel + 2
                ch0.append(np.squeeze(im_stack[ch0_idx,:,:]))
                ch1.append(np.squeeze(im_stack[ch1_idx,:,:]))


ch0_stack = np.stack(ch0, axis=0)    
ch1_stack = np.stack(ch1, axis=0)   
mc_stack = np.stack([ch0_stack, ch1_stack], axis=0)                 

In [31]:
#store as hdf5
outname = process_dir /  'traning_data_excl_last.hdf5'

h5f = h5py.File(outname, 'w')
h5f.create_dataset('dataset_1', data=mc_stack)
h5f.close()

In [34]:
#store as tiff
ch0n = process_dir / 'ph_training.tif'
ch1n = process_dir / 'gfp_training.tif'

tifffile.imwrite(ch0n, ch0_stack)
tifffile.imwrite(ch1n, ch1_stack)

In [ ]:
class register_pos:
    
    def __init__(self, pos, ch=0):
        self.pos = pos
        self.ch = ch
        self.transform = da.empty(0)
        self.reg_im = da.empty(0)
        return None
    
    def calc(self):
        self.transform = self._register_pos_calc().compute()
        self.reg_im = self._register_pos_apply()
        return reg_im  

    @dask.delayed
    def _get_shift(self, frm0, frm1):
        dxy, _, _ = phase_cross_correlation(np.squeeze(frm0),np.squeeze(frm1))
        return dxy

    @dask.delayed
    def _apply_shift(self, frm, dyx):
        return ndi.shift(frm, [0,*dyx], cval=0, order=0)

    def _register_pos_calc(self):
        shift = da.zeros((self.pos.shape[0],2), chunks=(1, 2))
        for t in range(self.pos.shape[0]-1):
            dxy = self._get_shift(self.pos[t,self.ch,:,:], self.pos[t+1,self.ch,:,:])
            shift[t+1,:] = da.from_delayed(dxy, shape=(1, 2), dtype=np.float64)
        return np.cumsum(shift, axis=0)     

    def _register_pos_apply(self):
        reg_im = da.empty_like(self.pos)
        for t in range(self.pos.shape[0]):
            im = self._apply_shift(self.pos[t,:,:,:], transform[t,:])
            reg_im[t,:,:,:] = da.from_delayed(im, shape=(self.pos.shape[1:]), dtype=np.float64)
        return reg_im